In [ ]:
!pip install pyomo==6.7.1

In [ ]:
from pathlib import Path

import pandas as pd
import datetime
import pyomo.environ as pe
import pyomo.gdp as pyogdp

from itertools import product
import numpy as np
import plotly.express as px

In [ ]:
####### EVALUATION #######
# TODO: check the values for commute and downtime counts to define the correct weight (currently it's 10)
# TODO: clean up preprocessing etc.
# TODO: run code for all days

####### QUESTION 2 #######
# TODO: filter for caregivers available that day
# TODO: change commute df to have bike time instead of driving time for no license
# TODO: filter / constrain for staff being able to perform task according to skills

####### QUESTION 3 #######
# TODO: create df sessions with additional for new personas -> do it for 3 different scenarios of combinations of personas or something
# TODO: run optimisation

####### "BONUS" #######
# TODO: add carbon emissions to objective function

In [ ]:
excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [ ]:
schedule = pd.read_excel(excel_file, sheet_name=0)
clients = pd.read_excel(excel_file, sheet_name=1)
caregivers = pd.read_excel(excel_file, sheet_name=2)

In [ ]:
schedule.Prestation.unique()

In [ ]:
discard_list = [
    "ADMINISTRATION",
    "VISITE MEDICALE",
    "FORMATION",
    "COORDINATION",
    "HOMMES TOUTES MAINS",
]

In [ ]:
sched_one_day = schedule.loc[schedule.Date == "2024-01-03", :]

In [ ]:
sched_one_day.Prestation.value_counts()

In [ ]:
sched_one_day = sched_one_day[~sched_one_day.Prestation.isin(discard_list)]

In [ ]:
sched_one_day["idx"] = sched_one_day.index

In [ ]:
from datetime import datetime

common_date = sched_one_day["Date"].iloc[0]
sched_one_day["Heure de fin"] = sched_one_day["Heure de fin"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Heure de début"] = sched_one_day["Heure de début"].apply(
    lambda x: datetime.combine(common_date, x)
)
sched_one_day["Duration"] = (
    sched_one_day["Heure de fin"] - sched_one_day["Heure de début"]
)
sched_one_day["Duration"] = sched_one_day["Duration"].apply(
    lambda x: x.seconds // 60
)

In [ ]:
sched_one_day

In [ ]:
before_first_client = caregivers[["ID Intervenant"]].copy()
before_first_client["ID Client"] = caregivers["ID Intervenant"]
before_first_client["Duration"] = 0
before_first_client["Date"] = pd.to_datetime("2024-01-03")
before_first_client["Heure de début"] = before_first_client[
    "Date"
] + pd.Timedelta(hours=5)
before_first_client["Heure de fin"] = before_first_client[
    "Date"
] + pd.Timedelta(hours=5)
before_first_client["Prestation"] = "COMMUTE"
before_first_client["idx"] = before_first_client.index

In [ ]:
after_last_client = caregivers[["ID Intervenant"]].copy()
after_last_client["ID Client"] = caregivers["ID Intervenant"]
after_last_client["Duration"] = 0
after_last_client["Date"] = pd.to_datetime("2024-01-03")
after_last_client["Heure de début"] = after_last_client["Date"] + pd.Timedelta(
    hours=22
)
after_last_client["Heure de fin"] = after_last_client["Date"] + pd.Timedelta(
    hours=22
)
after_last_client["Prestation"] = "COMMUTE"
after_last_client["idx"] = before_first_client.index + 10000

In [ ]:
sched_one_day = pd.concat(
    [sched_one_day, before_first_client], ignore_index=True
)
sched_one_day = pd.concat(
    [sched_one_day, after_last_client], ignore_index=True
)

In [ ]:
sched_one_day = sched_one_day.sort_values("Heure de début")
sched_one_day.to_csv("../data/schedule_2024-03-01.csv", index=False)

In [ ]:
clients_commute = pd.read_csv("../data/commute_driving_clients.csv")

In [ ]:
schedule["ID Client"].nunique()

In [ ]:
from itertools import product, combinations

len(
    list(
        product(schedule["ID Client"].unique(), schedule["ID Client"].unique())
    )
)

In [ ]:
len(
    list(
        product(
            schedule["ID Client"].unique(), schedule["ID Intervenant"].unique()
        )
    )
)

In [ ]:
len(list(combinations(schedule["ID Client"].unique(), 2)))

In [ ]:
clients_commute

In [ ]:
(clients_commute.source == clients_commute.destination).sum()

In [ ]:
list(
    clients_commute[["source", "destination", "commute_seconds"]].itertuples(
        index=False, name=None
    )
)

In [ ]:
# Loading and merging commute data
commute_file_paths = [
    "../data/commute_driving_clients.csv",
    "../data/commute_driving_care_clients.csv",
    "../data/commute_driving_clients_care.csv",
]


commute_dataframes = [pd.read_csv(file) for file in commute_file_paths]


for df in commute_dataframes:
    if df.columns[0] not in ["pair"]:  # standardizing column names
        df.rename(columns={df.columns[0]: "pair"}, inplace=True)

commute_data_df = pd.concat(commute_dataframes, ignore_index=True)

In [ ]:
commute_data_df[
    (commute_data_df.source == 522099746)
    & (commute_data_df.destination == 522099746)
]

In [ ]:
caregivers_commute = pd.DataFrame(
    {
        "source": schedule["ID Intervenant"].unique(),
        "destination": schedule["ID Intervenant"].unique(),
    }
)
caregivers_commute.insert(0, "commute_meters", 0)
caregivers_commute.insert(0, "commute_seconds", 0)
caregivers_commute.insert(
    0, "pair", list(zip(caregivers_commute.source, df.destination))
)
caregivers_commute["commute_method"] = "driving"

In [ ]:
commute_data_df = pd.concat([commute_data_df, caregivers_commute], axis=0)
commute_data_df.to_csv("../data/commute_driving_all.csv", index=False)

## Adding new commute dfs

In [ ]:
# Loading and merging commute data
commute_bicycling_file_paths = [
    "../data/commute_bicycling_clients.csv",
    "../data/commute_bicycling_care_clients.csv",
    "../data/commute_bicycling_clients_care.csv",
]


commute_bicycling_dataframes = [
    pd.read_csv(file) for file in commute_bicycling_file_paths
]


for df in commute_bicycling_dataframes:
    if df.columns[0] not in ["pair"]:  # standardizing column names
        df.rename(columns={df.columns[0]: "pair"}, inplace=True)

commute_bicycling_data_df = pd.concat(
    commute_bicycling_dataframes, ignore_index=True
)

In [ ]:
caregivers_commute = pd.DataFrame(
    {
        "source": schedule["ID Intervenant"].unique(),
        "destination": schedule["ID Intervenant"].unique(),
    }
)
caregivers_commute.insert(0, "commute_meters", 0)
caregivers_commute.insert(0, "commute_seconds", 0)
caregivers_commute.insert(
    0,
    "pair",
    list(zip(caregivers_commute.source, caregivers_commute.destination)),
)
caregivers_commute["commute_method"] = "bicycling"

In [ ]:
commute_bicycling_data_df = pd.concat(
    [commute_bicycling_data_df, caregivers_commute], axis=0
)
commute_bicycling_data_df.to_csv(
    "../data/commute_bicycling_all.csv", index=False
)

## Adding unavailable days for caregivers

In [ ]:
caregiver_avail_dict = {
    838320706: [1, 30, 31],
    609468992: [3, 10, 13, 14, 17, 24, 28, 31],
    78012267: [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18.19,
        20,
        21,
        22,
        23,
        24,
        26,
        27,
        28,
        29,
        30,
        31,
    ],
    818696864: [6, 7, 13, 14, 20, 21, 27, 28],
    746414886: [3, 5, 10, 17, 24, 31],
    78005437: [6, 7, 13, 14, 20, 21, 27, 28],
    213237245: [1, 2, 3, 13, 14, 16, 23, 27, 28, 30],
    813991780: [
        3,
        5,
        6,
        7,
        10,
        12,
        13,
        14,
        17,
        19,
        20,
        21,
        24,
        26,
        27,
        28,
        31,
    ],
    856089133: [
        5,
        9,
        11,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        31,
    ],
    700168298: [
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18.19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
    ],
    810259688: [5, 6, 7, 19, 20, 21, 28],
    78007018: [6, 13, 14, 20, 21, 31],
    764144581: [1, 10, 13, 14, 17, 24, 27, 28, 31],
    888888888: [
        1,
        3,
        4,
        5,
        6,
        7,
        8,
        10,
        17,
        19,
        20,
        21,
        22,
        24,
        26,
        27,
        29,
        30,
        31,
    ],
    854577575: [1, 2, 3, 8, 13, 14, 15, 22, 27, 28, 29],
    480302361: [6, 7, 15, 20, 21, 22, 29],
    78007874: [6, 7, 11, 13, 14, 18, 20, 21, 25, 27, 28],
    842621873: [1, 6, 7, 8, 9, 14, 15, 22, 27, 28, 29],
    710283561: [7, 8, 13, 14, 16, 23, 30],
    1452747150: [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        10,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        31,
    ],
    162858075: [5, 6, 7, 12, 19, 20, 21, 26],
    739888851: [
        1,
        3,
        5,
        6,
        7,
        8,
        10,
        12,
        13,
        14,
        15,
        19,
        20,
        21,
        22,
        24,
        26,
        27,
        28,
        29,
        31,
    ],
    648993440: [1, 2, 3, 4, 5, 6, 7, 11, 13, 18, 20, 21, 25],
    776399834: [
        1,
        2,
        6,
        7,
        9,
        10,
        13,
        14,
        16,
        17,
        20,
        21,
        22,
        25,
        26,
        27,
        28,
        30,
        31,
    ],
}

In [ ]:
caregiver_avail_df = pd.DataFrame(
    list(caregiver_avail_dict.items()),
    columns=["ID Intervenant", "UNDISP_DAYS"],
)
caregiver_avail_df.to_csv("../data/caregiver_avail.csv", index=False)

In [ ]:
import ast

caregiver_avail_df = pd.read_csv(
    "../data/caregiver_avail.csv",
    converters={
        "  ID Intervenant": ast.literal_eval,
        "UNDISP_DAYS": ast.literal_eval,
    },
)

In [ ]:
# try:
#            self.df_caregiver_avail = pd.read_csv('../data/caregiver_avail.csv', converters={"  ID Intervenant": ast.literal_eval,
#                                                         "UNDISP_DAYS": ast.literal_eval})
#        except FileNotFoundError:
#            print("Caregiver availability data not found")


#            self.day = self.df_sessions["Heure de début"].loc[0].day

In [ ]:
# def _generate_disjunctions(self):
#        """Returns:
#        disjunctions (list): list of tuples containing disjunctions
#        """
#        cases = self.df_sessions["idx"].to_list()
#        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
#        disjunctions = []
#        for (case1, case2, caregiver) in product(cases, cases, cargivers):
#            if (
#                self.df_sessions.loc[
#                    self.df_sessions["idx"] == case1, "ID Client"
#                ].iloc[0]
#                in cargivers
#            ) & (
#                self.df_sessions.loc[
#                    self.df_sessions["idx"] == case2, "ID Client"
#                ].iloc[0]
#                in cargivers
#            ):
#
#                if (
#                    self.df_sessions.loc[
#                        self.df_sessions["idx"] == case1, "ID Client"
#                    ].iloc[0]
#                    != caregiver
#                ) | (
#                    self.df_sessions.loc[
#                        self.df_sessions["idx"] == case2, "ID Client"
#                    ].iloc[0]
#                    != caregiver
#                ):
#                    continue
#
#            if (
#                self.df_sessions.loc[
#                    self.df_sessions["idx"] == case1, "ID Client"
#                ].iloc[0]
#                in cargivers
#            ) & (
#                not (
#                    self.df_sessions.loc[
#                        self.df_sessions["idx"] == case2, "ID Client"
#                    ].iloc[0]
#                    in cargivers
#                )
#            ):
#                if (
#                    self.df_sessions.loc[
#                        self.df_sessions["idx"] == case1, "ID Client"
#                    ].iloc[0]
#                    != caregiver
#                ):
#                    continue
#
#            if (
#                not (
#                    self.df_sessions.loc[
#                        self.df_sessions["idx"] == case1, "ID Client"
#                    ].iloc[0]
#                    in cargivers
#                )
#            ) & (
#                self.df_sessions.loc[
#                    self.df_sessions["idx"] == case2, "ID Client"
#                ].iloc[0]
#                in cargivers
#            ):
#            if (
#                self.df_sessions.loc[
#                    self.df_sessions["idx"] == case2, "ID Client"
#                ].iloc[0]
#                != caregiver
#                ):
#                    continue
#
#            if (case1 <= case2) & :
#                disjunctions.append((case1, case2, caregiver))
#
#        return disjunctions

In [ ]:
# def _generate_tasks(self):
#        cases = self.df_sessions["idx"].to_list()
#        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
#        tasks = []
#        for (case, caregiver) in product(cases, cargivers):
#            if (
#                self.df_sessions.loc[
#                    self.df_sessions["idx"] == case, "ID Client"
#                ].iloc[0]
#                in cargivers
#            ):
#                if (
#                    self.df_sessions.loc[
#                        self.df_sessions["idx"] == case, "ID Client"
#                    ].iloc[0]
#                    != caregiver
#                ):
#                    continue
#            if (self.day not in self.df_caregiver_avail.loc[self.df_caregiver_avail['ID Intervenant'] == caregiver, "UNDISP_DAYS"][0]):
#                tasks.append((case, caregiver))
#
#        return tasks

## Adding commute to how we calculate down time and commute time

In [ ]:
caregiver_transport = caregivers[["ID Intervenant", "Véhicule personnel"]]
caregiver_transport = caregiver_transport.fillna("Non")
caregiver_transport = caregiver_transport.replace({"Oui": True, "Non": False})
caregiver_transport.to_csv("../data/caregiver_transport.csv", index=False)

In [ ]:
caregiver_transport

In [ ]:
caregiver_transport.loc[
    caregiver_transport["ID Intervenant"] == 838320706, "Véhicule personnel"
][0]

In [ ]:
# try:
#            df_commute_bicycling = pd.read_csv("../data/commute_bicycling_all.csv")
#            df_commute_bicycling["commute_minutes"] = df_commute_bicycling["commute_seconds"] / 60
#            self.df_commute_bicycling = df_commute_bicycling
#        except FileNotFoundError:
#            print("Transit commute data not found")


# try:
#            df_caregiver_transport = pd.read_csv(""../data/caregiver_transport.csv", index=False")
#            self.df_caregiver_transport = df_caregiver_transport
#        except FileNotFoundError:
#            print("Caregiver transport data not found")

In [ ]:
def _generate_clients_commute_bicycling(self):
    clients_commute_bicycling = {}
    cargivers = self.df_cargeivers["ID Intervenant"].to_list()
    for source, dest in product(
        self.df_sessions["ID Client"].unique(),
        self.df_sessions["ID Client"].unique(),
    ):
        if (source in cargivers) and (dest in cargivers) and (source != dest):
            continue

        clients_commute_bicycling[
            (source, dest)
        ] = self.df_commute_bicycling.loc[
            (self.df_commute_bicycling.source == source)
            & (self.df_commute_bicycling.destination == dest),
            "commute_minutes",
        ].iloc[
            0
        ]
    return clients_commute_bicycling


# model.COMMUTE_BICYCLING = pe.Param(
#            model.CLIENT_CONNECTIONS,
#            initialize=self._generate_clients_commute_bicycling(),
#        )

In [ ]:
# def down_time_counts(model, case1, case2, caregiver):
#            if self.df_caregiver_transport.loc[self.df_caregiver_transport['ID Intervenant'] == caregiver, "Véhicule personnel"][0]:
#               commute_expr = (
#                model.SESSION_ASSIGNED[case1, case2, caregiver]
#                * int(
#                    (
#                        model.CASE_START_TIME[case2]
#                        - (
#                            model.CASE_START_TIME[case1]
#                            + model.CASE_DURATION[case1]
#                            + model.COMMUTE[
#                                (
#                                    model.IDX_CLIENTS[case1],
#                                    model.IDX_CLIENTS[case2],
#                                )
#                            ]
#                        )
#                    )
#                    < 30
#                )
#            )
#            else:
#                   commute_expr = (
#                model.SESSION_ASSIGNED[case1, case2, caregiver]
#                * int(
#                    (
#                        model.CASE_START_TIME[case2]
#                        - (
#                            model.CASE_START_TIME[case1]
#                            + model.CASE_DURATION[case1]
#                            + model.COMMUTE_BICYCLING[
#                                (
#                                    model.IDX_CLIENTS[case1],
#                                    model.IDX_CLIENTS[case2],
#                                )
#                            ]
#                        )
#                    )
#                    < 30
#                )
#            )
#
#            return model.DOWN_TIME_COUNTS[case1, case2, caregiver] == commute_expr

In [ ]:
#        def commute_care(model, case1, case2, caregiver):
#            if self.df_caregiver_transport.loc[self.df_caregiver_transport['ID Intervenant'] == caregiver, "Véhicule personnel"][0]:
#                commute_expr = model.SESSION_ASSIGNED[case1, case2, caregiver] * (
#                    model.COMMUTE[
#                        (
#                            model.IDX_CLIENTS[case1],
#                            model.IDX_CLIENTS[case2],
#                        )
#                    ]
#                )
#            else:
#                   commute_expr = model.SESSION_ASSIGNED[case1, case2, caregiver] * (
#                      model.COMMUTE_BICYCLING[
#                          (
#                              model.IDX_CLIENTS[case1],
#                              model.IDX_CLIENTS[case2],
#                          )
#                     ]
#                )
#            return model.COMMUTE_CARE[case1, case2, caregiver] == commute_expr
#
#        model.COMMUTE_CARE_CONST = pe.Constraint(
#            model.DISJUNCTIONS, rule=commute_care
#        )

In [ ]:
def no_case_overlap(model, case1, case2, caregiver):
    if model.df_caregiver_transport.loc[
        model.df_caregiver_transport["ID Intervenant"] == caregiver,
        "Véhicule personnel",
    ][0]:
        return [
            model.CASE_START_TIME[case1]
            + model.CASE_DURATION[case1]
            + model.COMMUTE[
                (model.IDX_CLIENTS[case1], model.IDX_CLIENTS[case2])
            ]
            <= model.CASE_START_TIME[case2]
            + (
                (1 - model.SESSION_ASSIGNED[case1, case2, caregiver]) * model.M
            ),
            model.CASE_START_TIME[case2]
            + model.CASE_DURATION[case2]
            + model.COMMUTE[
                (model.IDX_CLIENTS[case2], model.IDX_CLIENTS[case1])
            ]
            <= model.CASE_START_TIME[case1]
            + (
                (1 - model.SESSION_ASSIGNED[case1, case2, caregiver]) * model.M
            ),
        ]
    else:
        return [
            model.CASE_START_TIME[case1]
            + model.CASE_DURATION[case1]
            + model.COMMUTE_BICYCLING[
                (model.IDX_CLIENTS[case1], model.IDX_CLIENTS[case2])
            ]
            <= model.CASE_START_TIME[case2]
            + (
                (1 - model.SESSION_ASSIGNED[case1, case2, caregiver]) * model.M
            ),
            model.CASE_START_TIME[case2]
            + model.CASE_DURATION[case2]
            + model.COMMUTE_BICYCLING[
                (model.IDX_CLIENTS[case2], model.IDX_CLIENTS[case1])
            ]
            <= model.CASE_START_TIME[case1]
            + (
                (1 - model.SESSION_ASSIGNED[case1, case2, caregiver]) * model.M
            ),
        ]


#        model.DISJUNCTIONS_RULE = pyogdp.Disjunction(
#            model.DISJUNCTIONS, rule=no_case_overlap
#        )

In [ ]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            sessions = pd.read_csv(excel_file)

            sessions["idx"] = sessions.index

            sessions["Start_time"] = (
                pd.to_datetime(sessions["Heure de début"])
                - pd.to_datetime("2024-01-03 00:00:00")
            ).dt.seconds

            sessions["Start_time"] = sessions["Start_time"].apply(
                lambda x: x // 60
            )

            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel(
                "../data/ChallengeXHEC23022024.xlsx", sheet_name=2
            )
            # TODO: take kaans matrix -> filter cargeivers if they are not available today
        except FileNotFoundError:
            print("Caregiver data not found")

        try:
            df_commute = pd.read_csv("../data/commute_driving_all.csv")
            df_commute["commute_minutes"] = df_commute["commute_seconds"] / 60
            self.df_commute = df_commute
        except FileNotFoundError:
            print("Commute data not found")

        self.model = self.create_model()

    def _generate_case_durations(self):
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_start_time(self):
        return pd.Series(
            self.df_sessions["Start_time"].values,
            index=self.df_sessions["idx"],
        ).to_dict()

    def _generate_clients_commute(self):
        clients_commute = {}
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        for source, dest in product(
            self.df_sessions["ID Client"].unique(),
            self.df_sessions["ID Client"].unique(),
        ):
            if (
                (source in cargivers)
                and (dest in cargivers)
                and (source != dest)
            ):
                continue

            clients_commute[(source, dest)] = self.df_commute.loc[
                (self.df_commute.source == source)
                & (self.df_commute.destination == dest),
                "commute_minutes",
            ].iloc[0]
        return clients_commute

    def _IDX_CLIENTS_match(self):
        return pd.Series(
            self.df_sessions["ID Client"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, caregiver) in product(cases, cases, cargivers):
            if (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case1, "ID Client"
                ].iloc[0]
                in cargivers
            ) & (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case2, "ID Client"
                ].iloc[0]
                in cargivers
            ):

                if (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case1, "ID Client"
                    ].iloc[0]
                    != caregiver
                ) | (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case2, "ID Client"
                    ].iloc[0]
                    != caregiver
                ):
                    continue

            if (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case1, "ID Client"
                ].iloc[0]
                in cargivers
            ) & (
                not (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case2, "ID Client"
                    ].iloc[0]
                    in cargivers
                )
            ):
                if (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case1, "ID Client"
                    ].iloc[0]
                    != caregiver
                ):
                    continue

            if (
                not (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case1, "ID Client"
                    ].iloc[0]
                    in cargivers
                )
            ) & (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case2, "ID Client"
                ].iloc[0]
                in cargivers
            ):
                if (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case2, "ID Client"
                    ].iloc[0]
                    != caregiver
                ):
                    continue

            if case1 <= case2:
                disjunctions.append((case1, case2, caregiver))

        return disjunctions

    def _generate_tasks(self):
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        tasks = []
        for (case, caregiver) in product(cases, cargivers):
            if (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case, "ID Client"
                ].iloc[0]
                in cargivers
            ):
                if (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case, "ID Client"
                    ].iloc[0]
                    != caregiver
                ):
                    continue

            tasks.append((case, caregiver))

        return tasks

    def _case_combinations(self):
        cases = self.df_sessions["idx"].unique()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()

        case_comb = []
        for (case1, case2) in product(cases, cases):
            if (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case1, "ID Client"
                ].iloc[0]
                in cargivers
            ) & (
                self.df_sessions.loc[
                    self.df_sessions["idx"] == case2, "ID Client"
                ].iloc[0]
                in cargivers
            ):

                if (
                    self.df_sessions.loc[
                        self.df_sessions["idx"] == case1, "ID Client"
                    ].iloc[0]
                    != self.df_sessions.loc[
                        self.df_sessions["idx"] == case2, "ID Client"
                    ].iloc[0]
                ):
                    continue

            if case1 <= case2:
                case_comb.append((case1, case2))

        return case_comb

    def create_model(self):
        model = pe.ConcreteModel()

        # List of case IDs in home care client needs list
        model.CASES = pe.Set(initialize=self.df_sessions["idx"].tolist())
        # List of potential caregiver IDs
        model.CAREGIVERS = pe.Set(
            initialize=self.df_cargeivers["ID Intervenant"].tolist()
        )
        # Session utilisation
        model.DISJUNCTIONS = pe.Set(
            initialize=self._generate_disjunctions(), dimen=3
        )
        # List of tasks - all possible (caseID, caregiverID) combination
        model.TASKS = pe.Set(initialize=self._generate_tasks(), dimen=2)
        model.CASE_COMBINATIONS = pe.Set(
            initialize=self._case_combinations(),
            dimen=2,
        )

        # The duration (expected case time) for each operation
        model.CASE_DURATION = pe.Param(
            model.CASES, initialize=self._generate_case_durations()
        )
        # Start time of a case
        model.CASE_START_TIME = pe.Param(
            model.CASES, initialize=self._generate_start_time()
        )

        model.CLIENT_CONNECTIONS = pe.Set(
            initialize=product(
                self.df_sessions["ID Client"].unique(),
                self.df_sessions["ID Client"].unique(),
            )
        )
        model.IDX_CLIENTS = pe.Param(
            model.CASES, initialize=self._IDX_CLIENTS_match()
        )
        model.COMMUTE = pe.Param(
            model.CLIENT_CONNECTIONS,
            initialize=self._generate_clients_commute(),
        )

        # Decision Variables
        ub = 1440  # minutes in a day
        model.M = pe.Param(initialize=1e3 * ub)  # big M

        # Binary flag, 1 if case is assigned to session, 0 otherwise
        model.SESSION_ASSIGNED = pe.Var(model.DISJUNCTIONS, domain=pe.Binary)
        # commute for cargiver
        model.COMMUTE_CARE = pe.Var(
            model.DISJUNCTIONS, bounds=(0.0, 1440.0), within=pe.PositiveReals
        )
        model.DOWN_TIME_COUNTS = pe.Var(model.DISJUNCTIONS, within=pe.Binary)

        # Objective
        def objective_function(model):
            return pe.summation(model.COMMUTE_CARE) + 10 * pe.summation(
                model.DOWN_TIME_COUNTS
            )

        model.OBJECTIVE = pe.Objective(
            rule=objective_function, sense=pe.minimize
        )

        # each case can be maximum given once as source for all destinations and caregivers
        def session_assignment(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if case == case1
                    ]
                )
                <= 1
            )

        # each case can be maximum given once as destination for all sources and caregivers
        def session_assignment_2(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case2) & (case1 <= case2)
                    ]
                )
                <= 1
            )

        # each case needs to be given at least once as source or destination
        def session_assignment_3(model, case):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if case == case1
                    ]
                )
                + sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case2) & (case1 <= case2)
                    ]
                )
                >= 1
            )

        # if a case is assigned to a caregiver as source, it can't be assigned to a different caregiver as destination
        def session_assignment_4(model, case, caregiver_):
            return (
                sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case1) & (caregiver_ == caregiver)
                    ]
                )
                + sum(
                    [
                        model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                        for case1, case2, caregiver in model.DISJUNCTIONS
                        if (case == case2)
                        & (case1 <= case2)
                        & (caregiver_ != caregiver)
                    ]
                )
                <= 1
            )

        # if a case is assigned to a caregiver as destination, it also needs to be assigned as a source for this caregiver
        def session_assignment_6(model, case, caregiver_):
            return (
                (
                    sum(
                        [
                            model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                            for case1, case2, caregiver in model.DISJUNCTIONS
                            if (case == case1) & (caregiver_ == caregiver)
                            | (model.IDX_CLIENTS[case1] == caregiver)
                            & (caregiver_ == caregiver)
                        ]
                    )
                    - sum(
                        [
                            model.SESSION_ASSIGNED[(case1, case2, caregiver)]
                            for case1, case2, caregiver in model.DISJUNCTIONS
                            if (
                                (case == case2) & (caregiver_ == caregiver)
                                | (model.IDX_CLIENTS[case2] == caregiver)
                                & (caregiver_ == caregiver)
                            )
                            & (case1 <= case2)
                        ]
                    )
                )
            ) == 0

        model.SESSION_ASSIGNMENT = pe.Constraint(
            model.CASES, rule=session_assignment
        )
        model.SESSION_ASSIGNMENT_2 = pe.Constraint(
            model.CASES, rule=session_assignment_2
        )
        model.SESSION_ASSIGNMENT_3 = pe.Constraint(
            model.CASES, rule=session_assignment_3
        )
        model.SESSION_ASSIGNMENT_4 = pe.Constraint(
            model.TASKS, rule=session_assignment_4
        )
        model.SESSION_ASSIGNMENT_6 = pe.Constraint(
            model.TASKS, rule=session_assignment_6
        )

        def down_time_counts(model, case1, case2, caregiver):
            return model.DOWN_TIME_COUNTS[case1, case2, caregiver] == (
                model.SESSION_ASSIGNED[case1, case2, caregiver]
                * int(
                    (
                        model.CASE_START_TIME[case2]
                        - (
                            model.CASE_START_TIME[case1]
                            + model.CASE_DURATION[case1]
                            + model.COMMUTE[
                                (
                                    model.IDX_CLIENTS[case1],
                                    model.IDX_CLIENTS[case2],
                                )
                            ]
                        )
                    )
                    < 30
                )
            )

        model.DOWNTIME_CNTS = pe.Constraint(
            model.DISJUNCTIONS, rule=down_time_counts
        )

        def commute_care(model, case1, case2, caregiver):
            commute_expr = model.SESSION_ASSIGNED[case1, case2, caregiver] * (
                model.COMMUTE[
                    (
                        model.IDX_CLIENTS[case1],
                        model.IDX_CLIENTS[case2],
                    )
                ]
            )
            return model.COMMUTE_CARE[case1, case2, caregiver] == commute_expr

        model.COMMUTE_CARE_CONST = pe.Constraint(
            model.DISJUNCTIONS, rule=commute_care
        )

        def no_case_overlap(model, case1, case2, caregiver):
            return [
                model.CASE_START_TIME[case1]
                + model.CASE_DURATION[case1]
                + model.COMMUTE[
                    (model.IDX_CLIENTS[case1], model.IDX_CLIENTS[case2])
                ]
                <= model.CASE_START_TIME[case2]
                + (
                    (1 - model.SESSION_ASSIGNED[case1, case2, caregiver])
                    * model.M
                ),
                model.CASE_START_TIME[case2]
                + model.CASE_DURATION[case2]
                + model.COMMUTE[
                    (model.IDX_CLIENTS[case2], model.IDX_CLIENTS[case1])
                ]
                <= model.CASE_START_TIME[case1]
                + (
                    (1 - model.SESSION_ASSIGNED[case1, case2, caregiver])
                    * model.M
                ),
            ]

        model.DISJUNCTIONS_RULE = pyogdp.Disjunction(
            model.DISJUNCTIONS, rule=no_case_overlap
        )

        pe.TransformationFactory("gdp.bigm").apply_to(model)

        return model

    def solve(self):
        # solvername = "glpk"
        solvername = "cbc"

        # solverpath_folder = (
        #     "C:\\glpk\\w64"  # does not need to be directly on c drive
        # )

        # solverpath_exe = (
        #     "C:\\glpk\\w64\\glpsol"  # does not need to be directly on c drive
        # )
        solverpath_exe = "/opt/homebrew/bin/cbc"
        solver = pe.SolverFactory(solvername, executable=solverpath_exe)

        # Add solver parameters (time limit)
        options = {"seconds": 120}
        for key, value in options.items():
            solver.options[key] = value

        # Solve model (verbose)
        solver_results = solver.solve(self.model, tee=True)
        return solver_results

In [ ]:
scheduler = CareScheduler("../data/schedule_2024-03-01.csv")

In [ ]:
solver_results = scheduler.solve()

In [ ]:
model = scheduler.model

In [ ]:
model.OBJECTIVE.display()

In [ ]:
actions = [
    k for k, v in model.SESSION_ASSIGNED.extract_values().items() if v == 1
]

In [ ]:
sorted(set(list(model.SESSION_ASSIGNED.extract_values().values())))

In [ ]:
actions_df = pd.DataFrame(actions, columns=["idx1", "idx2", "Caregiver_ID"])

In [ ]:
actions_df_1 = actions_df[["idx1", "Caregiver_ID"]]
actions_df_2 = actions_df[["idx2", "Caregiver_ID"]]
actions_df_1.columns = ["idx", "Caregiver_ID"]
actions_df_2.columns = ["idx", "Caregiver_ID"]

In [ ]:
actions_df = pd.concat([actions_df_1, actions_df_2], axis=0)
actions_df = actions_df.drop_duplicates()

In [ ]:
temp = scheduler.df_sessions.copy()
temp = temp.merge(actions_df, how="left", on="idx")
temp

In [ ]:
temp.idx.unique()

In [ ]:
temp.Caregiver_ID.nunique()

In [ ]:
temp.Caregiver_ID.isna().mean()

In [ ]:
temp.style

## Plot optimised schedule

In [ ]:
jan24_df = temp.copy()
jan24_df = jan24_df[jan24_df.Prestation != "COMMUTE"]

In [ ]:
# Loading and merging commute data
commute_file_paths = [
    "../data/commute_bicycling_clients.csv",
    "../data/commute_driving_clients.csv",
    "../data/commute_bicycling_care_clients.csv",
    "../data/commute_bicycling_clients_care.csv",
    "../data/commute_driving_care_clients.csv",
    "../data/commute_driving_clients_care.csv",
]


commute_dataframes = [pd.read_csv(file) for file in commute_file_paths]


for df in commute_dataframes:
    if df.columns[0] not in ["pair"]:  # standardizing column names
        df.rename(columns={df.columns[0]: "pair"}, inplace=True)

commute_data_df = pd.concat(commute_dataframes, ignore_index=True)

In [ ]:
caregivers["Commute Method"] = caregivers["Véhicule personnel"].map(
    {"Oui": "driving", "Non": "bicycling", np.nan: "bicycling"}
)  # map commute method
jan24_df = jan24_df.merge(
    caregivers[["ID Intervenant", "Commute Method"]],
    left_on="Caregiver_ID",
    right_on="ID Intervenant",
    how="left",
)  # merge with agenda data

jan24_df["Start DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de début"].astype(str)
)
jan24_df["End DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de fin"].astype(str)
)

commute_data_df[["source", "destination"]] = commute_data_df[
    "pair"
].str.extract(r"\((\d+), (\d+)\)")
commute_data_df.drop(columns="pair", inplace=True)
commute_data_df.set_index(
    ["source", "destination", "commute_method"], inplace=True
)

In [ ]:
def compute_commute_and_wait_times(df, commute_data_df):
    df["Wait Time"] = 0
    df["Commute Time"] = 0

    for intervenant_id in df["ID Intervenant"].unique():
        intervenant_data = df[df["ID Intervenant"] == intervenant_id]

        for date in intervenant_data["Date"].unique():
            daily_data = intervenant_data[
                intervenant_data["Date"] == date
            ].sort_values(by="Start DateTime")
            prev_end_time = None
            prev_client_id = None

            for index, row in daily_data.iterrows():
                destination_id = str(row["ID Client"])
                commute_method = "driving"  # row['Commute Method'] if 'Commute Method' in row else 'driving'

                if prev_client_id is None or prev_end_time is None:
                    source_id = str(intervenant_id)
                else:
                    source_id = str(prev_client_id)

                    wait_time = (
                        row["Start DateTime"] - prev_end_time
                    ).total_seconds() // 60 - commute_data_df.loc[
                        (source_id, destination_id, commute_method),
                        "commute_seconds",
                    ] // 60
                    if wait_time < 30:
                        df.loc[index, "Wait Time"] = wait_time

                try:
                    commute_time = (
                        commute_data_df.loc[
                            (source_id, destination_id, commute_method),
                            "commute_seconds",
                        ]
                        // 60
                    )
                except KeyError:
                    commute_time = 0  # Default to 0 if not found
                    print(
                        f"Data not found for commute time: {source_id}, {destination_id}, {commute_method}"
                    )

                df.loc[index, "Commute Time"] = commute_time

                # Update previous end time and client ID for next iteration
                prev_end_time = row["End DateTime"]
                prev_client_id = row["ID Client"]

    return df

In [ ]:
def plot_agenda(intervenant_id, jan24_df, commute_data_df):
    intervenant_agenda = jan24_df[jan24_df["ID Intervenant"] == intervenant_id]
    intervenant_agenda_sorted = intervenant_agenda.sort_values(
        by=["Date", "Heure de début"]
    )

    df_timeline = intervenant_agenda_sorted.copy()
    df_timeline["Start"] = pd.to_datetime(df_timeline["Start DateTime"])
    df_timeline["Finish"] = pd.to_datetime(df_timeline["End DateTime"])
    df_timeline["Task"] = df_timeline["Prestation"]
    df_timeline["Resource"] = df_timeline["ID Intervenant"].astype(str)
    df_timeline["ID Client"] = df_timeline["ID Client"].astype(str)

    df_timeline = compute_commute_and_wait_times(df_timeline, commute_data_df)

    commute_entries = df_timeline.copy()
    commute_entries["Start"] = commute_entries["Start"] - pd.to_timedelta(
        commute_entries["Commute Time"], unit="m"
    )
    commute_entries["Finish"] = commute_entries["Start"] + pd.to_timedelta(
        commute_entries["Commute Time"], unit="m"
    )
    commute_entries["Task"] = "Commute Time"
    commute_entries["Type"] = "Commute"

    wait_entries = []
    prev_finish = None
    for _, row in df_timeline.iterrows():
        if prev_finish is not None and row["Wait Time"] > 0:
            wait_entry = row.copy()
            wait_entry["Start"] = prev_finish
            wait_entry["Finish"] = row["Start"]
            wait_entry["Task"] = "Wait Time"
            wait_entry["Type"] = "Wait"
            wait_entries.append(wait_entry)
        prev_finish = row["Finish"]
    wait_entries_df = pd.DataFrame(wait_entries)

    df_timeline["Type"] = "Task"

    end_of_day_commutes = []
    for date in df_timeline["Date"].unique():
        daily_data = df_timeline[df_timeline["Date"] == date]
        if not daily_data.empty:
            last_client_id = daily_data.iloc[-1]["ID Client"]
            source_id = str(last_client_id)
            destination_id = str(intervenant_id)
            commute_method = daily_data.iloc[-1]["Commute Method"]

            try:
                commute_time = (
                    commute_data_df.loc[
                        (source_id, destination_id, commute_method),
                        "commute_seconds",
                    ]
                    / 60
                )
            except KeyError:
                print("Commute time not found")

            end_of_day_commute = {
                "Start": daily_data.iloc[-1]["End DateTime"],
                "Finish": daily_data.iloc[-1]["End DateTime"]
                + pd.Timedelta(minutes=commute_time),
                "Task": "Commute Time",
                "Type": "Commute",
                "ID Client": "Home",
                "Date": date,
                "ID Intervenant": intervenant_id,
                "Commute Time": commute_time,
                "Commute Method": commute_method,
                "Wait Time": 0,
            }
            end_of_day_commutes.append(end_of_day_commute)

    end_of_day_commutes_df = pd.DataFrame(end_of_day_commutes)

    combined_df = pd.concat(
        [df_timeline, commute_entries, wait_entries_df, end_of_day_commutes_df]
    )
    combined_df.sort_values(by="Start", inplace=True)

    fig = px.timeline(
        combined_df,
        x_start="Start",
        x_end="Finish",
        y="Task",
        color="Type",
        color_discrete_map={
            "Task": "blue",
            "Commute": "orange",
            "Wait": "green",
        },
        hover_data=["ID Client", "Wait Time", "Commute Time"],
    )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=f"Agenda for Intervenant ID: {intervenant_id}")
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(
                            count=1,
                            label="1D",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(
                            count=7,
                            label="1W",
                            step="day",
                            stepmode="backward",
                        ),
                        dict(step="all"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        )
    )

    fig.show()

    return combined_df

In [ ]:
all_intervenant_agendas = []

for intervenant_id in jan24_df["ID Intervenant"].unique():
    intervenant_agenda_commute = plot_agenda(
        intervenant_id, jan24_df, commute_data_df
    )
    all_intervenant_agendas.append(intervenant_agenda_commute)

df_agendas = pd.concat(all_intervenant_agendas)

In [ ]:
df_agendas[(df_agendas["Task"] == "Commute Time")].groupby("ID Intervenant")[
    "Commute Time"
].sum()

In [ ]:
df_agendas.loc[(df_agendas["Task"] == "Commute Time"), "Commute Time"].sum()

In [ ]:
df_agendas[(df_agendas["Task"] == "Wait Time")].groupby("ID Intervenant")[
    "Wait Time"
].count()

In [ ]:
df_agendas.loc[(df_agendas["Task"] == "Wait Time"), "Wait Time"].count()

### Compare to given schedule

In [ ]:
jan24_df = sched_one_day.copy()
jan24_df = jan24_df[jan24_df.Prestation != "COMMUTE"]

caregivers["Commute Method"] = caregivers["Véhicule personnel"].map(
    {"Oui": "driving", "Non": "bicycling", np.nan: "bicycling"}
)  # map commute method
jan24_df = jan24_df.merge(
    caregivers[["ID Intervenant", "Commute Method"]],
    on="ID Intervenant",
    how="left",
)  # merge with agenda data

jan24_df["Start DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de début"].astype(str)
)
jan24_df["End DateTime"] = pd.to_datetime(
    jan24_df["Date"].astype(str) + " " + jan24_df["Heure de fin"].astype(str)
)

In [ ]:
all_intervenant_agendas = []

for intervenant_id in jan24_df["ID Intervenant"].unique():
    intervenant_agenda_commute = plot_agenda(
        intervenant_id, jan24_df, commute_data_df
    )
    all_intervenant_agendas.append(intervenant_agenda_commute)

df_agendas = pd.concat(all_intervenant_agendas)

In [ ]:
df_agendas[(df_agendas["Task"] == "Commute Time")].groupby("ID Intervenant")[
    "Commute Time"
].sum()

In [ ]:
df_agendas.loc[(df_agendas["Task"] == "Commute Time"), "Commute Time"].sum()

In [ ]:
df_agendas[(df_agendas["Task"] == "Wait Time")].groupby("ID Intervenant")[
    "Wait Time"
].count()

In [ ]:
df_agendas.loc[(df_agendas["Task"] == "Wait Time"), "Wait Time"].count()